In [1]:
from collections import OrderedDict, defaultdict, Counter
import pandas as pd
import csv
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
data = pd.read_csv("../data/clean/scores.csv").drop("Unnamed: 0", axis=1)
lexicon = '../resources/lexicons/NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt'

In [3]:
def create_emotionList(lexicon):
    wordList = defaultdict(list)
    emotionList = defaultdict(list)
    with open(lexicon, 'r') as f:
        reader = csv.reader(f, delimiter='\t')
        headerRows = [i for i in range(0, 46)]
        for row in headerRows:
            next(reader)
        for word, emotion, present in reader:
            if int(present) == 1:
                #print(word)
                wordList[word].append(emotion)
                emotionList[emotion].append(word)
    return wordList

In [4]:
wordList = create_emotionList(lexicon)
tt = TweetTokenizer()

In [5]:
def generate_emotion_count(string, tokenizer):
    emoCount = Counter()
    for token in tt.tokenize(string):
        token = token.lower()
        emoCount += Counter(wordList[token])
    #dividing by length of string   
    for k in emoCount.keys():
        emoCount[k] = emoCount[k]*1.0/len(tt.tokenize(string))
        
    return emoCount

In [6]:
def add_emotion_score(data):
    emotion_scores = [generate_emotion_count(tweet, tt) for tweet in data['text']]
    emotion_df = pd.DataFrame(emotion_scores, index=data.index)
    emotion_df = emotion_df.fillna(0)
    return pd.concat([data, emotion_df], axis = 1)

In [22]:
d = add_emotion_score(data)
d.drop(['score_keyword', 'score_textblob_bayes_sentiment', 'score_textblob_pattern_sentiment', 'text', 'score'], axis=1, inplace=True)
d.to_csv(path_or_buf ="../data/clean/scores_emotion.csv")

In [23]:
print(d.shape)
print(len(d.id_str.unique()))
d.head(1)

(294, 11)
294


,id_str,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,921829947093733376,0.03125,0.03125,0.0,0.03125,0.03125,0.09375,0.03125,0.0,0.03125,0.03125
